# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [10]:
# Install pyspark
!pip install pyspark
# import findspark
# findspark.init()
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
You are working with 1 core(s)


In [129]:
path = '/content/drive/MyDrive/Data Science Intake43/5. Spark/spark-scripts/section2/Datasets/'
os.listdir(path)

['people.json',
 'zomato.csv',
 'Weather.csv',
 'youtubevideos.csv',
 'googleplaystore.csv',
 'fifa19.csv',
 'students.csv',
 'users1.parquet',
 'users3.parquet',
 'users2.parquet',
 'nyc_air_bnb.csv',
 'rec-crime-pfa.csv',
 'supermarket_sales.csv',
 'Rep_vs_Dem_tweets.csv',
 'pga_tour_historical.csv',
 'uw-madison-courses']

## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [160]:
df = spark.read.csv(path+'Rep_vs_Dem_tweets.csv',inferSchema=True,header=True)
# data shape
print((df.count(), len(df.columns)))

(92489, 3)


## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [161]:
df.limit(10).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
5,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...
6,Democrat,RepDarrenSoto,RT @EmgageActionFL: Thank you to all who came ...
7,Democrat,RepDarrenSoto,Hurricane Maria left approx $90 billion in dam...
8,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...
9,Democrat,RepDarrenSoto,RT @HispanicCaucus: Trump's anti-immigrant pol...


**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [162]:
df.show(5, False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [163]:
df.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [164]:
# Create a temporary view of the dataframe
df.createOrReplaceTempView("tempview")

In [165]:
df.where(df['Tweet'].contains('@LatinoLeader')).show(10, False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+



In [166]:
from pyspark.sql.functions import *
df.select(regexp_extract(df['Tweet'], '(.)(@LatinoLeader)(.)', 2)).show(20)

+-----------------------------------------------+
|regexp_extract(Tweet, (.)(@LatinoLeader)(.), 2)|
+-----------------------------------------------+
|                                               |
|                                               |
|                                               |
|                                           null|
|                                  @LatinoLeader|
|                                               |
|                                               |
|                                               |
|                                               |
|                                               |
|                                               |
|                                               |
|                                               |
|                                               |
|                                               |
|                                           null|
|                                               |


## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [167]:
from pyspark.sql.functions import *
df = df.withColumn('Party', when(df.Party.isin('Democrat', 'Republican'), df.Party).otherwise('other'))
df.show(10)

+--------+-------------+--------------------+
|   Party|       Handle|               Tweet|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|Today, Senate Dem...|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|   other|         null|                null|
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|Democrat|RepDarrenSoto|RT @EmgageActionF...|
|Democrat|RepDarrenSoto|Hurricane Maria l...|
|Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|Democrat|RepDarrenSoto|RT @HispanicCaucu...|
+--------+-------------+--------------------+
only showing top 10 rows



In [168]:
df.select('Tweet').show(10,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

In [169]:
df.select('Party').distinct().collect()

[Row(Party='Republican'), Row(Party='Democrat'), Row(Party='other')]

## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [170]:
# (https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)
df = df.withColumn("no_links",regexp_replace("Tweet","(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)",""))
df.select('no_links').show(10, False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|no_links                                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

## 4. Remove any leading or trailing white space in the tweet column

In [171]:
df.columns

['Party', 'Handle', 'Tweet', 'no_links']

In [172]:
# [ \t]+$
df = df.withColumn("no_white_spaces",regexp_replace("no_links","[ \t]+$",""))
df = df.withColumn("no_white_spaces", trim(df.no_white_spaces))
df.select('no_white_spaces').show(10, False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|no_white_spaces                                                                                                                             |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

In [173]:
df.columns

['Party', 'Handle', 'Tweet', 'no_links', 'no_white_spaces']

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [174]:
df = df.withColumnRenamed("Party", "Dem_Rep")
df.limit(4).toPandas()

,Dem_Rep,Handle,Tweet,no_links,no_white_spaces
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...","Today, Senate Dems vote to #SaveTheInternet. P...","Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,RT @WinterHavenSun: Winter Haven resident / Al...,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,other,None,None,None,None


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [175]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [176]:
df.select(df.Dem_Rep,df.Handle,concat_ws(" ", df.Dem_Rep, df.Handle).alias("Concatenated")).show(5, False)

+--------+-------------+----------------------+
|Dem_Rep |Handle       |Concatenated          |
+--------+-------------+----------------------+
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|other   |null         |other                 |
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
+--------+-------------+----------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [177]:
hashtags = spark.sql("""
    select  regexp_extract_all(Tweet, '(#\\\\w+)', 1) as a 
    from    tempview
    where   Tweet like '%#%'
""")

hashtags.show(truncate=False)

+-----------------------------------+
|a                                  |
+-----------------------------------+
|[#SaveTheInternet, #NetNeutrality] |
|[#NALCABPolicy2018]                |
|[#NetNeutrality]                   |
|[#Orlando]                         |
|[#HFAnow, #paidsickdays]           |
|[#classroomwhereithappens]         |
|[#FACT]                            |
|[#ABetterDeal]                     |
|[#Science, #Sayfie]                |
|[#2020Census]                      |
|[#ghostguns, #9Investigates, #wftv]|
|[#APAHM]                           |
|[#Insulin4all]                     |
|[#EmgagetheHill, #EmgageAction]    |
|[#Emgagethehill]                   |
|[#TPS, #Hondurans]                 |
|[#HurricaneMaria]                  |
|[#vamos4pr]                        |
|[#DemsAtWor]                       |
|[#DemsAtWork]                      |
+-----------------------------------+
only showing top 20 rows



# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [178]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [179]:
df = df.withColumn("dob", df["dob"].cast(DateType())).withColumn("visit1", df["visit1"].cast(DateType())).withColumn("visit2", df["visit2"].cast(DateType())).withColumn("visit3", df["visit3"].cast(DateType()))

print(df.printSchema())

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

None


In [180]:
df.show(10,False)

+----------+---------+----------+----------+----------+----------+
|first_name|last_name|dob       |visit1    |visit2    |visit3    |
+----------+---------+----------+----------+----------+----------+
|Mohammed  |Alfasy   |1987-04-08|2016-01-07|2017-02-03|2018-03-02|
|Marcy     |Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|
|Ginny     |Ginger   |1986-07-10|2014-08-07|2015-02-03|2016-03-02|
|Vijay     |Doberson |1988-05-02|2016-01-07|2018-02-03|2018-03-02|
|Orhan     |Gelicek  |1987-05-11|2016-05-07|2017-01-03|2018-09-02|
|Sarah     |Jones    |1956-07-06|2016-04-07|2017-08-03|2018-10-02|
|John      |Johnson  |2017-10-12|2018-01-02|2018-10-03|2018-03-02|
+----------+---------+----------+----------+----------+----------+



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [181]:
df = df.withColumn("vis1_2", datediff(df.visit2, df.visit1).alias("diff1_2"))
df.select('visit1', 'visit2','vis1_2').show(5,False)
print('Avarage length between the first 2 visits:')
df.agg({'vis1_2': 'avg'}).show(10,False)

+----------+----------+------+
|visit1    |visit2    |vis1_2|
+----------+----------+------+
|2016-01-07|2017-02-03|393   |
|2015-01-07|2017-01-03|727   |
|2014-08-07|2015-02-03|180   |
|2016-01-07|2018-02-03|758   |
|2016-05-07|2017-01-03|241   |
+----------+----------+------+
only showing top 5 rows

Avarage length between the first 2 visits:
+------------------+
|avg(vis1_2)       |
+------------------+
|436.57142857142856|
+------------------+



In [182]:
df = df.withColumn("vis1_3", datediff(df.visit3, df.visit1).alias("diff1_3"))
df.select('visit1', 'visit3','vis1_3').show(5,False)
print('Avarage length between the first and third visits:')
df.agg({'vis1_3': 'avg'}).show(10,False)

+----------+----------+------+
|visit1    |visit3    |vis1_3|
+----------+----------+------+
|2016-01-07|2018-03-02|785   |
|2015-01-07|2018-01-02|1091  |
|2014-08-07|2016-03-02|573   |
|2016-01-07|2018-03-02|785   |
|2016-05-07|2018-09-02|848   |
+----------+----------+------+
only showing top 5 rows

Avarage length between the first and third visits:
+-----------------+
|avg(vis1_3)      |
+-----------------+
|721.2857142857143|
+-----------------+



## 8. Can you calculate the age of each patient?

In [183]:
df = df.withColumn('age', floor(datediff(current_date(), to_date(col('dob'), 'M/d/yyyy'))/365.25))
df.select('dob','age').show(10, False)

+----------+---+
|dob       |age|
+----------+---+
|1987-04-08|35 |
|1986-04-08|36 |
|1986-07-10|36 |
|1988-05-02|34 |
|1987-05-11|35 |
|1956-07-06|66 |
|2017-10-12|5  |
+----------+---+



## 9. Can you extract the month from the visit columns and call it "Month"?

In [184]:
df = df.withColumn('month', month(df['visit1']).alias('month'))
df.select('visit1','month').show(3)

+----------+-----+
|    visit1|month|
+----------+-----+
|2016-01-07|    1|
|2015-01-07|    1|
|2014-08-07|    8|
+----------+-----+
only showing top 3 rows



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [185]:
df = spark.read.csv(path + "supermarket_sales.csv", inferSchema=True, header=True)
# data shape
print((df.count(), len(df.columns)))

(1000, 17)


## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [186]:
df.limit(6).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,2023-03-05 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,2023-03-05 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,2023-03-05 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,2023-03-05 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,2023-03-05 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3
5,699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,3/25/2019,2023-03-05 18:30:00,Ewallet,597.73,4.761905,29.8865,4.1


In [187]:
df.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [191]:
from pyspark.sql.types import *

print("This gives all null values")
df = df.withColumn("New_Date", to_date(df.Date, "m/d/y"))
df.select('Date', 'New_Date').show(10, False)

This gives all null values
+---------+----------+
|Date     |New_Date  |
+---------+----------+
|1/5/2019 |2019-01-05|
|3/8/2019 |2019-01-08|
|3/3/2019 |2019-01-03|
|1/27/2019|2019-01-27|
|2/8/2019 |2019-01-08|
|3/25/2019|2019-01-25|
|2/25/2019|2019-01-25|
|2/24/2019|2019-01-24|
|1/10/2019|2019-01-10|
|2/20/2019|2019-01-20|
+---------+----------+
only showing top 10 rows



### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [192]:
df.select(month('New_Date')).show()

+---------------+
|month(New_Date)|
+---------------+
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
+---------------+
only showing top 20 rows



## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [193]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [197]:
reviews = reviews.withColumn("cleaned", trim(lower(regexp_replace(col("review_txt"), "[^\sa-zA-Z0-9]", ""))))
reviews.show(10, False)

+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                            |cleaned                                                                            |
+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |epic this is the best movie i have ever seen                                       |
|4     |Pretty good, but I would have liked to seen better special effects                    |pretty good but i would have liked to seen better special effects                  |
|3     |So so. Casting could have been improved                                               |so so

In [198]:
reviews = reviews.withColumn("splitted", split(col("cleaned"), " "))
reviews.show(10, False)

+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |cleaned                                                                            |splitted                                                                                           |
+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |epic this is the best movie i have ever seen                     

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [202]:
reviews = reviews.withColumn("contains_epic", lower(reviews["review_txt"]).contains('epic'))
reviews.limit(5).toPandas()

,rating,review_txt,cleaned,splitted,contains_epic
0,5,Epic. This is the best movie I have EVER seen,epic this is the best movie i have ever seen,"[epic, this, is, the, best, movie, i, have, ev...",True
1,4,"Pretty good, but I would have liked to seen be...",pretty good but i would have liked to seen bet...,"[pretty, good, but, i, would, have, liked, to,...",False
2,3,So so. Casting could have been improved,so so casting could have been improved,"[so, so, casting, could, have, been, improved]",False
3,5,The most EPIC movie of the year! Casting was a...,the most epic movie of the year casting was aw...,"[the, most, epic, movie, of, the, year, castin...",True
4,4,Solid but I would have liked to see more of th...,solid but i would have liked to see more of th...,"[solid, but, i, would, have, liked, to, see, m...",False


In [204]:
reviews.filter('contains_epic = True').show(10, False)

+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+-------------+
|rating|review_txt                                                                            |cleaned                                                                            |splitted                                                                                           |contains_epic|
+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+-------------+
|5     |Epic. This is the best movie I have EVER seen                                         |epic this is the best m

### That's it! Great Job!